# ReAct Agent Demo: Data & Math Tools
Este notebook demonstra um ReAct Agent com ferramentas de cálculo de datas e expressões matemáticas, usando OpenAI Functions.

In [ ]:
# Instalação da biblioteca OpenAI
!pip install openai

In [ ]:
import os, json, logging, openai
from typing import Callable, List, Dict, Any

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Tool:
    def __init__(self, name: str, description: str, func: Callable[..., Any], parameters_schema: Dict[str, Any]):
        self.name = name
        self.description = description
        self.func = func
        self.parameters_schema = parameters_schema

    def to_openai_function(self) -> Dict[str, Any]:
        return {
            "name": self.name,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": self.parameters_schema,
                "required": list(self.parameters_schema.keys())
            }
        }

class ReActAgent:
    def __init__(self, tools: List[Tool], system_prompt: str, model: str="gpt-4-0613", max_steps: int=5):
        self.model = model
        self.tools = {tool.name: tool for tool in tools}
        self.system_prompt = system_prompt
        self.max_steps = max_steps

    def run(self, query: str) -> str:
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": query}
        ]
        functions = [tool.to_openai_function() for tool in self.tools.values()]

        for step in range(1, self.max_steps + 1):
            logger.info(f"[Step {step}/{self.max_steps}] Enviando ao LLM...")
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=messages,
                functions=functions,
                function_call="auto"
            )
            msg = response.choices[0].message

            if msg.get("function_call"):
                name = msg["function_call"]["name"]
                args = json.loads(msg["function_call"]["arguments"])
                logger.info(f"Chamando {name} com args {args}")
                try:
                    result = self.tools[name].func(**args)
                except Exception as e:
                    result = f"Error: {e}"
                messages.append({"role": "assistant", "content": None, "function_call": msg["function_call"]})
                messages.append({"role": "function", "name": name, "content": json.dumps(result)})
                continue

            content = msg.get("content")
            if content:
                logger.info("Resposta final recebida.")
                return content.strip()

        return "🛑 Max steps reached"

# Configurar chave de API
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import datetime

def days_until(date_str: str) -> Dict[str, Any]:
    from datetime import datetime
    target = datetime.strptime(date_str, "%Y-%m-%d")
    now = datetime.now()
    delta = target - now
    return {"days": delta.days, "weekday": target.strftime("%A")}

def calculate(expression: str) -> float:
    return eval(expression)

In [ ]:
# Registro das ferramentas
tools = [
    Tool(
        name="days_until",
        description="Retorna dias até a data e o dia da semana dessa data.",
        func=days_until,
        parameters_schema={
            "date_str": {"type": "string", "description": "Data no formato YYYY-MM-DD"}
        }
    ),
    Tool(
        name="calculate",
        description="Avalia uma expressão matemática e retorna o resultado.",
        func=calculate,
        parameters_schema={
            "expression": {"type": "string", "description": "Expressão a ser avaliada."}
        }
    )
]

In [ ]:
# Prompt do sistema
system_prompt = """
You are a ReAct agent using a Thought → Action → Observation loop.
- Thought: descreva seu raciocínio.
- Action: chame uma das ferramentas com argumentos JSON.
- Observation: será fornecido o resultado da ferramenta.
- Quando finalizado, responda com Answer: sua resposta final.

Available tools:
- days_until(date_str): retorna {"days": int, "weekday": str}.
- calculate(expression): avalia expressões matemáticas.

Example:
Question: How many days until 2025-12-25, then square that number?
Thought: I need days until 2025-12-25.
Action: days_until("2025-12-25")
Observation: {"days": 250, "weekday": "Friday"}
Thought: Now calculate 250 ** 2.
Action: calculate("250 ** 2")
Observation: 62500
Answer: 62500
"""

In [ ]:
# Executando o agente com a pergunta de demonstração
agent = ReActAgent(tools, system_prompt, model="gpt-4-0613", max_steps=7)
question = "How many days until 2025-12-25, then square that value?"
answer = agent.run(question)
print("Question:", question)
print("Answer:", answer)